# 1번 레이어 : 합성곱층 (convolutional layer)
합성곱(in_channel = 1, out_channel =32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2)

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2)

# 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size x 7 x 7 x 64 -> batch_size x 3136
전결합층(뉴런 10개) + 활서오하 함수 Softmax

In [2]:
import torch
import torch.nn as nn

In [4]:
inputs= torch.Tensor(1,1,28,28)
inputs.size()

torch.Size([1, 1, 28, 28])

In [6]:
conv1=nn.Conv2d(1,32,3,padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [8]:
conv2=nn.Conv2d(32,64,3, padding=1)
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [9]:
pool=nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [10]:
out=conv1(inputs)
print(out.size())

torch.Size([1, 32, 28, 28])


In [11]:
out=pool(out)
print(out.size())

torch.Size([1, 32, 14, 14])


In [14]:
out=conv2(out)
print(out.size())


RuntimeError: Given groups=1, weight of size [64, 32, 3, 3], expected input[1, 64, 14, 14] to have 32 channels, but got 64 channels instead

In [15]:
out=pool(out)
print(out.size())

torch.Size([1, 64, 7, 7])


In [16]:
out=out.view(out.size(0),-1)
print(out.size())

torch.Size([1, 3136])


In [17]:
fc=nn.Linear(3136,10)
out=fc(out)
print(out.size())

torch.Size([1, 10])


In [18]:
import torch
import torch.nn.init
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#랜덤 시드 고정
torch.manual_seed(777)
#gpu 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
    

In [20]:
learning_rate=0.001
training_epochs = 30
batch_size = 100

In [21]:
mnist_train=dsets.FashionMNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(),
                        download=True)
mnist_test=dsets.FashionMNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(),
                        download=True)

100%|██████████| 26421880/26421880 [00:06<00:00, 3978290.95it/s]


Extracting MNIST_data/FashionMNIST\raw\train-images-idx3-ubyte.gz to MNIST_data/FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 113923.63it/s]


Extracting MNIST_data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to MNIST_data/FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:02<00:00, 1632042.87it/s]


Extracting MNIST_data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to MNIST_data/FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<00:00, 5147145.89it/s]

Extracting MNIST_data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to MNIST_data/FashionMNIST\raw



In [22]:
from torch.utils.data import DataLoader


In [24]:
data_loader=DataLoader(dataset=mnist_train, #훈련데이터
                       batch_size=batch_size, # 배치의 크기
                       shuffle=True, # 매 에포크마다 미니배치를 서플할 것인가?
                       drop_last=True #마지막 배치를 버릴 것인가?
                       )


In [28]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        #1 layer image :(?,28,28,1),conv:(?,28,28,32), pool :(?,14,14,32)
        self.layer1=nn.Sequential(
            nn.Conv2d(1,32,3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        #2 layer image :(?,14,14,32),conv:(?,14,14,64), pool :(?,7,7,64)
        self.layer2=nn.Sequential(
            nn.Conv2d(32,64,3,stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc=nn.Linear(7*7*64,10,bias=True) # 완전연결층 7*7*64 -> 10

        nn.init.xavier_uniform_(self.fc.weight) # 완전연결층 한정 가중치 초기화
    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0), -1)
        out=self.fc(out)
        return out

In [30]:

model=CNN().to(device)
criterion=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [32]:
total_batch=len(data_loader)
print("총 배치 수 : {}".format(total_batch))

총 배치 수 : 600


In [33]:
for epoch in range(training_epochs):
    avg_cost=0

    for X,Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        y_hat=model(X)
        cost=criterion(y_hat,Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/total_batch

    print("Epoch={:>4}  cost={:>.9f}".format(epoch+1, avg_cost))

Epoch=   1  cost=0.494553208
Epoch=   2  cost=0.323722333
Epoch=   3  cost=0.282947451
Epoch=   4  cost=0.258439064
Epoch=   5  cost=0.239216357
Epoch=   6  cost=0.224087417
Epoch=   7  cost=0.209569260
Epoch=   8  cost=0.198688999
Epoch=   9  cost=0.186568588
Epoch=  10  cost=0.175413728
Epoch=  11  cost=0.166557789
Epoch=  12  cost=0.156520352
Epoch=  13  cost=0.149516940
Epoch=  14  cost=0.143661559
Epoch=  15  cost=0.134650484
Epoch=  16  cost=0.125849903
Epoch=  17  cost=0.119643122
Epoch=  18  cost=0.114492245
Epoch=  19  cost=0.107492961
Epoch=  20  cost=0.101298161
Epoch=  21  cost=0.096533887
Epoch=  22  cost=0.092822455
Epoch=  23  cost=0.088990986
Epoch=  24  cost=0.083473690
Epoch=  25  cost=0.079350241
Epoch=  26  cost=0.076074570
Epoch=  27  cost=0.070626967
Epoch=  28  cost=0.067128301
Epoch=  29  cost=0.063911311
Epoch=  30  cost=0.061985414


# 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2))

# 3번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 64, out_channel = 128, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2, padding=1))

# 4번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 128, out_channel = 256, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
맥스풀링(kernel_size=2, stride=2, padding=1))

# 5번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size × 4 × 4 × 256 → batch_size × 2048
전결합층(뉴런 625개) + 활성화 함수 ReLU

# 6번 레이어 : 전결합층(Fully-Connected layer)
전결합층(뉴런 10개) + 활성화 함수 Softmax

In [34]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print("Accuracy:", accuracy.item())

c:\Users\admin\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\admin\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.7695000171661377


In [ ]:
mnist_train = dsets.CIFAR10(root='cifar10_data/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

mnist_test = dsets.CIFAR10(root='cifar10_data/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)